In [23]:
import os
# import h5py
from util.misc import saveraster
from j_output.user_output import TmrtOutput
# from j_dataprep import DEMs

from collections import defaultdict
# from j_dataprep.landcover import LandCover
from osgeo import gdal
# from j_dataprep.DEMs import Buildings
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
import pandas as pd

# for average heat day no trees

In [5]:
bbox_dict = {
        'historisch': [(175905, 317210, 176505, 317810), (84050, 447180, 84650, 447780), (80780, 454550, 81380, 455150),
                       (233400, 581500, 234000, 582100), (136600, 455850, 137200, 456450),
                       (121500, 487000, 122100, 487600)
                       ],
        'tuindorp': [(76800, 455000, 78200, 455700), (152600, 463250, 153900, 463800), (139140, 469570, 139860, 470400),
                     (190850, 441790, 191750, 442540), (113100, 551600, 113650, 552000), (32050, 391900, 32850, 392500)

                     ],
        'vinex': [(146100, 486500, 147000, 487400), (153750, 467550, 154650, 468450), (115300, 517400, 116100, 518250),
                  (102000, 475900, 103100, 476800), (160750, 388450, 161650, 389350), (84350, 449800, 85250, 450700)

                  ],
        'volkswijk': [(104200, 490550, 105100, 491450), (78200, 453900, 79100, 454800), (83500, 447020, 84050, 447900),
                      (136200, 456500, 137100, 457300), (182700, 579200, 183800, 579750),
                      (233400, 582800, 234300, 583700)

                      ],
        'bloemkool': [(81700, 427490, 82700, 428200), (84050, 444000, 84950, 444900), (116650, 518700, 117550, 519600),
                      (235050, 584950, 235950, 585850), (210500, 473900, 211400, 474800),
                      (154700, 381450, 155700, 382150)

                      ],

        'stedelijk': [
            (90300, 436900, 91300, 437600), (91200, 438500, 92100, 439300), (121350, 483750, 122250, 484650),
            (118400, 486400, 119340, 487100)
        ]
}

gpkg_names = ['typos', "tuindorp", 'vinexwijk', 'volkswijk', 'bloemkoolwijk', 'stedelijk_bouwblok']

base_folder = "E:/Geomatics/thesis/_analysisfinalfurther/"
typologies = ['historisch', 'tuindorp', 'vinex', 'volkswijk', 'bloemkool', 'stedelijk']
locs = [0, 1, 2, 3, 4, 5]



In [ ]:

for typ in typologies:
    typology_path = os.path.join(base_folder, typ)
    for i in locs:
        try:
            boundingbox = bbox_dict[typ][i]
        except (KeyError, IndexError):
            print(f"Skipping {typ}, index {i} – bounding box not found.")
            continue

        dtm_path = typology_path + f"/loc_{i}/final_dtm.tif"

        buildingslocation = Buildings(boundingbox)


        landcover = LandCover(boundingbox, dataset_path=dtm_path, building_data=buildingslocation.building_geometries)

        building_mask = ~landcover.building_mask
        water_mask = landcover.water_mask

        if water_mask is not None:
            total_mask = np.logical_or(building_mask, ~water_mask).astype(np.uint8)
        else:
            total_mask = building_mask.astype(np.uint8)

        dataset = gdal.Open(dtm_path)
        saveraster(dataset, typology_path + f'/loc_{i}/masks.tif', total_mask)

In [16]:
gpkg_path = "E:/Geomatics/thesis/_typos/typos.gpkg"

j = 0

for typ in typologies:
    typology_path = os.path.join(base_folder, typ)

    layer_name = gpkg_names[j]
    gdf = gpd.read_file(gpkg_path, layer=layer_name)

    for i in locs:
        try:
            boundingbox = bbox_dict[typ][i]
        except (KeyError, IndexError):
            print(f"Skipping {typ}, index {i} – bounding box not found.")
            continue

        dtm_path = typology_path + f"/loc_{i}/final_dtm.tif"
        with rasterio.open(dtm_path) as src:
            dtm_meta = src.meta.copy()
            transform = src.transform
            crs = src.crs
            shape = (src.height, src.width)

        dataset = gdal.Open(dtm_path)
        file = dataset.GetRasterBand(1).ReadAsArray()

        body = np.zeros_like(file)

        geom_mask = rasterize(
            [(geom, 1) for geom in gdf.geometry],
            out_shape=shape,
            transform=transform,
            fill=0,
            dtype='uint8'
        )
        new_mask = (geom_mask == 0).astype(np.uint8)

        existing_mask_gdal = gdal.Open(f'{typology_path}/loc_{i}/masks.tif')
        existing_mask = existing_mask_gdal.ReadAsArray()

        total_mask = np.logical_or(existing_mask, new_mask).astype(np.uint8)

        saveraster(dataset, typology_path + f'/loc_{i}/masks_combined.tif', total_mask)

    j += 1

Skipping stedelijk, index 4 – bounding box not found.
Skipping stedelijk, index 5 – bounding box not found.


In [34]:
base_folder = "E:/Geomatics/thesis/_analysisfinalfurther"
typologies = ['historisch', 'tuindorp', 'vinex', 'volkswijk', 'bloemkool', 'stedelijk']
locs = [0, 1, 2, 3, 4, 5]
sol = 'solweig_ext'


rows = []

for typ in typologies:
    typology_path = os.path.join(base_folder, typ)

    for i in locs:
        try:
            boundingbox = bbox_dict[typ][i]
        except (KeyError, IndexError):
            print(f"Skipping {typ}, index {i} – bounding box not found.")
            continue

        area_path = typology_path + f"/loc_{i}/{sol}/"
        print(area_path)
        mask_path = typology_path + f'/loc_{i}/masks_combined.tif'

        mask_gdal = gdal.Open(mask_path)
        mask = ~mask_gdal.ReadAsArray().astype(bool)

        tmrt_processor = TmrtOutput(output_folder=area_path, building_mask=mask)
        print(tmrt_processor.tmrt_arrays_by_time)

        for time_group, avg_array in tmrt_processor.averaged_tmrt.items():
            if avg_array is not None:
                stats = tmrt_processor.calculate_stats_and_bins(avg_array, isTmrt=True)
                print(stats)


E:/Geomatics/thesis/_analysisfinalfurther\historisch/loc_0/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\historisch/loc_1/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\historisch/loc_2/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\historisch/loc_3/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\historisch/loc_4/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\historisch/loc_5/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\tuindorp/loc_0/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\tuindorp/loc_1/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\tuindorp/loc_2/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\tuindorp/loc_3/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\tuindorp/loc_4/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\tuindorp/loc_5/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\vinex/loc_0/solweig_ext/
{}
E:/Geomatics/thesis/_analysisfinalfurther\

In [29]:
print(rows)

[]


In [26]:
df = pd.DataFrame(rows)

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

# Filter for bin range of interest, e.g. (30, 35) °C
target_range = (30, 35)
df_range = df[df['bin_range'] == target_range]

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_range, x='typology', y='bin_percentage', hue='time_group')
plt.title(f"Tmrt bin {target_range} °C: % Area per Typology")
plt.ylabel("Percentage of Area")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

KeyError: 'bin_range'